# Calculate wind capacity factors for NEM region using 20min BARRA-C2

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [2]:
walltime = "01:00:00"
cores = 48
memory = str(4 * cores) + "GB"

cluster = PBSCluster(
    walltime=str(walltime),
    cores=cores,
    memory=str(memory),
    processes=cores,
    job_extra_directives=[
        "-q normal",
        "-P dt6",
        "-l ncpus="+str(cores),
        "-l mem="+str(memory),
        "-l storage=gdata/w42+gdata/rt52+gdata/ob53+scratch/w42"
        # "-l storage=gdata/xp65"
    ],
    local_directory="$TMPDIR",
    job_directives_skip=["select"],
    # python="/g/data/xp65/public/apps/med_conda_scripts/analysis3-25.08.d/bin/python",
    # job_script_prologue=['module load conda/analysis3-25.08'],
    log_directory="/scratch/w42/dr6273/tmp/logs"
)

/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35903 instead
  warnings.warn(


In [3]:
cluster.scale(jobs=1)
client = Client(cluster)

In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/35903/status,
Dashboard: /proxy/35903/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.121.2:39819,Workers: 0
Dashboard: /proxy/35903/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt

In [8]:
barra_path = "/g/data/ob53/BARRA2/output/reanalysis/AUST-04/BOM/ERA5/historical/hres/BARRA-C2/v1/20min/"
write_path = "/scratch/w42/dr6273/BARRA-C2/"

In [8]:
# # Box encompassing NEM REZs
# lons = slice(133.5, 153.7)
# lats = slice(-43.4, -13.9)
lons = fn.get_rez_boundary()["lon"]
lats = fn.get_rez_boundary()["lat"]

### Compute wind speed

In [9]:
def windspeed(u, v):
    """
    Compute windspeed from u and v
    
    u: array of zonal wind
    v: array of meridional wind
    """
    return np.sqrt(u ** 2 + v ** 2)

In [10]:
# def capacity_factor_vdW(W):
#     """
#     Computes capacity factor from wind speed data.
    
#     W: wind speed (m/s)
#     """
#     W_0 = 3.5 # cut-in speed (m/s)
#     W_r = 13 # rated speed
#     W_1 = 25 # cut-out speed (m/s)
    
#     # Cubic
#     c_f = (W ** 3 - W_0 ** 3) / (W_r ** 3 - W_0 ** 3)
#     c_f = c_f.where(W >= W_0, 0) # Set values below cut-in to zero
#     c_f = c_f.where(W < W_r, 1) # Set values above rated speed to 1
#     c_f = c_f.where(W < W_1, 0) # Set values above cut-off to zero
#     c_f = c_f.where(W.notnull(), np.nan) # Ensure NaNs are retained
    
#     return c_f

In [11]:
def open_barra(fp, lat_slice, lon_slice, lat_name="lat", lon_name="lon"):
    """
    Open multiple files and preprocess to region.
    
    fp: str, path to file. Should not include files, only the path to dir.
    lat_slice, lon_slice: slice of lat/lon to subset
    lat_name, lon_name: names of lat/lon coords.
    """
    def preprocess(ds):
        # ds = ds.rename({lat_name: "lat"})
        # ds = ds.rename({lon_name: "lon"})
        ds = ds.sel(lon=lon_slice, lat=lat_slice)
        # ds["time"] = ds["time"].dt.round(freq="20min")
        return ds.astype("float32")
    
    ds = xr.open_mfdataset(
        fp,
        preprocess=preprocess,
        chunks={"time": "300MB"},# "lat": -1, "lon": -1},
        concat_dim="time",
        combine="nested",
        compat="override",
        coords="minimal",
        data_vars="minimal",
        # decode_times=True
        # parallel=True
    )
    return ds

In [12]:
def get_filename(variable, frequency, year, month):
    """
    Return BARRA-C2 AUST-04 historical data filepaths
    
    variable: str, variable name e.g. ua100m
    frequency: str, timestep e.g. 20min, 1hr
    year: str
    month: str, month in format '01', '02', ..., '12'
    """
    file_template = variable + "_AUST-04_ERA5_historical_hres_BOM_BARRA-C2_v1_" + frequency + "_"
    return barra_path + variable + "/latest/" + file_template + year + month + "-" + year + month + ".nc"

In [13]:
years = range(1979, 2025)
freq = "20min"
height_str = "100m"
month_str = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

In [18]:
years[41:]

range(2020, 2025)

In [19]:
# client.restart()

In [19]:
for year in years[41:]:
    if year % 5 == 0:
        print(year)
        
    monthly_arrays = []
    for month in month_str[:]:
        
        u = open_barra(
            get_filename("ua"+height_str, freq, str(year), month),
            lats,
            lons
        )
        v = open_barra(
            get_filename("va"+height_str, freq, str(year), month),
            lats,
            lons
        )
        w = windspeed(
            u.rename({"ua"+height_str: "w"+height_str}),
            v.rename({"va"+height_str: "w"+height_str})
        )
        # w = w.chunk({"time": 200, "lat": -1, "lon": -1})
        # w = w.persist()
        
        cf = fn.capacity_factor_vdW(w["w"+height_str])
                
        # Chunk size of 72 as it is the greatest common divisor of
        #. the different time steps associated with each month
        cf = cf.chunk({"time": 72, "lat": -1, "lon": -1})
        
        cf = cf.to_dataset(name="cf"+height_str)
        
        encoding = {
            "cf"+height_str: {"dtype": "float32"}
        }
        
        if month == "01":
            cf.to_zarr(
                write_path + "derived/wind_capacity_factor/cf" + height_str + "_vdW_BARRA-C2_" + freq + "_" + str(year) + ".zarr",
                mode="w",
                encoding=encoding,
                consolidated=True,
            )
        else:
            cf.to_zarr(
                write_path + "derived/wind_capacity_factor/cf" + height_str + "_vdW_BARRA-C2_" + freq + "_" + str(year) + ".zarr",
                mode="a",
                append_dim="time",
                consolidated=True,
            )

2020


# Close cluster

In [59]:
client.close()
cluster.close()